### Feature Extraction and Audio Classification

In [7]:
import librosa
import pandas as pd
import os
audio_path='Audio_Data/Urban_sound_data/'
metadata=pd.read_csv(r'Audio_Data/Urban_sound_data/UrbanSound8k.csv')
metadata.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [8]:
def features_extractor(filename):
    audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features

In [9]:
import numpy as np
from tqdm import tqdm
feature_list=[]
for index_num,row in tqdm(metadata.iterrows()):
    filename=os.path.join(os.path.abspath(audio_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    final_class_labels=row["class"]
    data=features_extractor(filename)
                       

3555it [07:48,  8.78it/s]C:\Users\Satya\anaconda3\lib\site-packages\librosa\core\spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=1323
  n_fft, y.shape[-1]
8326it [17:07, 11.34it/s]C:\Users\Satya\anaconda3\lib\site-packages\librosa\core\spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=1103
  n_fft, y.shape[-1]
C:\Users\Satya\anaconda3\lib\site-packages\librosa\core\spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=1523
  n_fft, y.shape[-1]
8732it [17:49,  8.17it/s]


In [52]:
feature_list.append([data,final_class_labels])
feature_list_df=pd.DataFrame(feature_list,columns=['Features','class'])
feature_list_df.head()

,Features,class
0,"[-315.93384, 95.67589, -38.047768, 47.50074, -...",car_horn
1,"[-315.93384, 95.67589, -38.047768, 47.50074, -...",car_horn
2,"[-315.93384, 95.67589, -38.047768, 47.50074, -...",car_horn
3,"[-315.93384, 95.67589, -38.047768, 47.50074, -...",car_horn
4,"[-315.93384, 95.67589, -38.047768, 47.50074, -...",car_horn


In [53]:
x=np.array(feature_list_df['Features'].tolist())
y=np.array(feature_list_df['class'].tolist())

In [54]:
x.shape

(7, 40)

In [55]:
y=np.array(pd.get_dummies(y))

In [56]:
y.shape

(7, 1)

In [57]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)


In [58]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(5, 40)
(2, 40)
(5, 1)
(2, 1)


In [ ]:
!pip uninstall tensorflow

In [ ]:
!pip install tensorflow==2.0.0

In [59]:
import tensorflow as tf
print(tf.__version__)

2.0.0


In [60]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [61]:
num_labels=y.shape[1]

In [32]:
#Dense()

In [62]:
model=Sequential()
###first layer
model.add(Dense(100,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###second layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###third layer
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))

###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [63]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 100)               4100      
_________________________________________________________________
activation_9 (Activation)    (None, 100)               0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 200)               20200     
_________________________________________________________________
activation_10 (Activation)   (None, 200)               0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 100)              

In [64]:
model.compile(loss='categorical_crossentropy',metrics=['Accuracy'],optimizer='adam')

In [67]:
## Trianing my model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_model\audio_classification.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

ValueError: You are passing a target array of shape (5, 1) while using as loss `categorical_crossentropy`. `categorical_crossentropy` expects targets to be binary matrices (1s and 0s) of shape (samples, classes). If your targets are integer classes, you can convert them to the expected format via:
```
from keras.utils import to_categorical
y_binary = to_categorical(y_int)
```

Alternatively, you can use the loss function `sparse_categorical_crossentropy` instead, which does expect integer targets.

In [ ]:
test_accuracy=model.evaluate(x_test,y_test,verbose=0)
print(test_accuracy[1])

In [ ]:
filename="UrbanSound8K/dog_bark.wav"
predict_feature=features_extractor(filename)
predict_feature=predict_feature.reshape(1,-1)
predict_class=model.predict_classes('predict_feature')
print(predict_class)